In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import pandas as pd
from upath import UPath

### EIA

In [3]:
eia_df = pd.read_parquet("gs://sec10k-outputs/v2/core_eia__parents_and_subsidiaries.parquet")

### Ex. 21

In [3]:
ex21_df = pd.read_pickle("/Users/katielamb/CatalystCoop/dagster_home/storage/transformed_ex21_subsidiary_table")

In [6]:
ex21_df

,sec_company_id,company_name_raw,location_of_inc,own_per,filename,report_date,report_year,company_name,company_name_no_legal,company_name_mphone,parent_company_cik
0,0000000020_colormax limited_united kingdom,colormax limited,united kingdom,NaN,edgar/data/20/0000893220-06-000650.txt,2006-03-23,2006,colormax limited,colormax,KLRMKS,0000000020
1,0000000020_gundlach equipment corporation_dela...,gundlach equipment corporation,delaware,NaN,edgar/data/20/0000950123-10-024631.txt,2010-03-15,2010,gundlach equipment corporation,gundlach equipment,KNTLX EKPMNT,0000000020
2,0000000020_jeffrey rader ab_sweden,jeffrey rader ab,sweden,NaN,edgar/data/20/0000950123-10-024631.txt,2010-03-15,2010,jeffrey rader ab,jeffrey rader ab,JFR RTR AB,0000000020
3,0000000020_jeffrey rader canada company_canada,jeffrey rader canada company,canada,NaN,edgar/data/20/0000950123-10-024631.txt,2010-03-15,2010,jeffrey rader canada company,jeffrey rader canada,JFR RTR KNT,0000000020
4,0000000020_jeffrey rader corporation_delaware,jeffrey rader corporation,delaware,NaN,edgar/data/20/0000950123-10-024631.txt,2010-03-15,2010,jeffrey rader corporation,jeffrey rader,JFR RTR,0000000020
...,...,...,...,...,...,...,...,...,...,...,...
1055982,0001967649_vestis supply chain limited liabili...,"vestis (supply chain), llc",delaware,NaN,edgar/data/1967649/0001967649-23-000025.txt,2023-12-21,2023,vestis supply chain limited liability company,vestis supply chain,FSTS SPL XN,0001967649
1055983,0001967649_vestis syracuse limited liability c...,"vestis (syracuse), llc",delaware,NaN,edgar/data/1967649/0001967649-23-000025.txt,2023-12-21,2023,vestis syracuse limited liability company,vestis syracuse,FSTS SRKS,0001967649
1055984,0001967649_vestis texas limited liability comp...,"vestis (texas), llc",delaware,NaN,edgar/data/1967649/0001967649-23-000025.txt,2023-12-21,2023,vestis texas limited liability company,vestis texas,FSTS TKSS,0001967649
1055985,0001967649_vestis west adams limited liability...,"vestis (west adams), llc",delaware,NaN,edgar/data/1967649/0001967649-23-000025.txt,2023-12-21,2023,vestis west adams limited liability company,vestis west adams,FSTS WST ATMS,0001967649


In [25]:
merged_df = ex21_df.merge(
        eia_df.drop_duplicates(subset="company_name")[["company_name", "utility_id_eia"]], how="left", on="company_name", suffixes=("_ex21", "_eia")
    )

In [14]:
len(ex21_df), len(eia_df)

(1055987, 20821)

In [27]:
merged_df.utility_id_eia.isnull().value_counts()

utility_id_eia
True     1050887
False       5100
Name: count, dtype: int64

In [29]:
len(merged_df.utility_id_eia.unique())

1675

### Read in SEC output table

In [26]:
# review outputs from Dagster
sec_out_df = pd.read_parquet(UPath("gs://sec10k-outputs/v2/out_sec_10k__parents_and_subsidiaries/2023q1.parquet"))

In [27]:
sec_out_df

,sec_company_id,filename,business_phone,central_index_key,city,company_name,date_of_name_change,film_number,fiscal_year_end,form_type,...,street_1,street_2,zip,report_date,report_year,location_of_inc,company_name_clean,parent_company_cik,own_per,files_10k
0,0000001800,edgar/data/1800/0001628280-23-004026.txt,2246676100,0000001800,abbott park,abbott laboratories,None,23642562,1231,10-k,...,100 abbott park road,None,60064-3500,2023-02-17,2023,illinois,abbott laboratories,None,None,True
1,0000001800_3a nutrition (vietnam) company limi...,edgar/data/1800/0001628280-23-004026.txt,None,None,None,3a nutrition (vietnam) company limited,None,None,None,None,...,None,None,None,2023-02-17,2023,viet nam,3a nutrition vietnam company limited,0000001800,None,False
2,"0000001800_abbott (jiaxing) nutrition co., ltd...",edgar/data/1800/0001628280-23-004026.txt,None,None,None,"abbott (jiaxing) nutrition co., ltd",None,None,None,None,...,None,None,None,2023-02-17,2023,china,abbott jiaxing nutrition co limited,0000001800,None,False
3,0000001800_abbott (shanghai) diagnostics sales...,edgar/data/1800/0001628280-23-004026.txt,None,None,None,"abbott (shanghai) diagnostics sales co., ltd",None,None,None,None,...,None,None,None,2023-02-17,2023,china,abbott shanghai diagnostics sales co limited,0000001800,None,False
4,0000001800_abbott (uk) finance limited_united ...,edgar/data/1800/0001628280-23-004026.txt,None,None,None,abbott (uk) finance limited,None,None,None,None,...,None,None,None,2023-02-17,2023,united kingdom,abbott uk finance limited,0000001800,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171358,0001951118,edgar/data/1951118/0001853620-23-000117.txt,(248) 991-6700,0001951118,farmington hills,mercedes-benz auto receivables trust 2022-1,None,23764946,1231,10-k,...,35555 w. twelve mile rd.,suite 100,48331,2023-03-27,2023,delaware,mercedes benz auto receivables trust 2022 1,None,None,True
171359,0001951752,edgar/data/1951752/0001951752-23-000016.txt,3135943495,0001951752,dearborn,ford credit auto owner trust 2022-d,None,23751556,1231,10-k,...,"c/o ford motor co , whq ste 801-c1",one american road,48126,2023-03-22,2023,None,ford credit auto owner trust 2022 d,None,None,True
171360,0001954336,edgar/data/1477336/0001954336-23-000024.txt,313-656-5500,0001954336,wilmington,ally auto receivables trust 2022-3,None,23759320,1231,10-k,...,1209 orange street,None,19801,2023-03-24,2023,delaware,ally auto receivables trust 2022 3,None,None,True
171361,0001954436,edgar/data/1954436/0000929638-23-001050.txt,(214) 572-8276,0001954436,irving,exeter automobile receivables trust 2022-6,None,23784761,1231,10-k,...,2101 w. john carpenter freeway,None,75063,2023-03-31,2023,delaware,exeter automobile receivables trust 2022 6,None,None,True


### There are a combination of SEC 10K filers and subsidiary companies:

In [28]:
sec_out_df.files_10k.value_counts()

files_10k
False    165824
True       5539
Name: count, dtype: int64

### `sec_company_id` and `central_index_key` should be unique:

In [29]:
sec_out_df.sec_company_id.is_unique

True

In [30]:
sec_out_df.central_index_key.dropna().is_unique

True

### Location of incorporation should be clean and standardized for filers and subsidiaries.

In [32]:
sec_out_df[sec_out_df.files_10k]["location_of_inc"].value_counts().head(10)

location_of_inc
delaware          3076
nevada             300
maryland           299
cayman islands     135
north carolina      92
new york            74
florida             74
pennsylvania        71
california          57
texas               56
Name: count, dtype: int64

In [33]:
sec_out_df[sec_out_df.files_10k]["location_of_inc"].value_counts().tail(10)

location_of_inc
bahamas                  1
germany                  1
hong kong                1
china                    1
virgin islands, u.s.     1
quebec, canada           1
new brunswick, canada    1
new hampshire            1
netherlands antilles     1
malaysia                 1
Name: count, dtype: int64

In [34]:
sec_out_df[~sec_out_df.files_10k]["location_of_inc"].value_counts().head(10)

location_of_inc
delaware          67546
united kingdom     4979
cayman islands     3000
texas              2881
netherlands        2615
california         2566
germany            2381
china              2305
florida            2130
australia          1938
Name: count, dtype: int64

In [35]:
sec_out_df[~sec_out_df.files_10k]["location_of_inc"].value_counts().tail(10)

location_of_inc
ontario, can                 1
british col, can             1
hong kong china china        1
zhongshan, china             1
jacksonville, florida        1
toronto, ontario, canada     1
limassol, cyprus             1
doncaster, syorkshire, uk    1
manchester, england          1
cote                         1
Name: count, dtype: int64

### All non SEC 10K filers should have a `parent_company_cik`

In [36]:
sec_out_df[~sec_out_df.files_10k]["parent_company_cik"].isnull().value_counts()

parent_company_cik
False    165824
Name: count, dtype: int64

### When run on all year quarters, all `parent_company_cik` should appear in `central_index_key` column

In [45]:
n_parent_company_cik = len(set(sec_out_df.parent_company_cik))
n_parent_company_cik

2954

In [46]:
n_overlap = len(set(sec_out_df.parent_company_cik).intersection(set(sec_out_df.central_index_key)))
n_overlap

2832

In [47]:
n_parent_company_cik - n_overlap

122

### There should be filer companies that have a `parent_company_cik` because they were matched to a subsidiary

In [49]:
sec_out_df[sec_out_df.files_10k].parent_company_cik.isnull().value_counts()

parent_company_cik
True     5474
False      65
Name: count, dtype: int64

### There should be no non-filer companies that have a CIK

In [50]:
sec_out_df[~sec_out_df.files_10k].central_index_key.isnull().value_counts()

central_index_key
True    165824
Name: count, dtype: int64

### There should be no duplicated `company_name`, `location_of_inc`, `parent_company_cik` records

In [52]:
sec_out_df[sec_out_df.duplicated(subset=["company_name", "location_of_inc", "parent_company_cik"])]

,sec_company_id,filename,business_phone,central_index_key,city,company_name,date_of_name_change,film_number,fiscal_year_end,form_type,...,street_1,street_2,zip,report_date,report_year,location_of_inc,company_name_clean,parent_company_cik,own_per,files_10k


### There can be companies with the same name, location, and CIK, but different parent companies.

In [56]:
sec_out_df = sec_out_df.fillna({"central_index_key": pd.NA})

In [59]:
sec_out_df[~sec_out_df.central_index_key.isnull() 
           & (sec_out_df.duplicated(
               subset=["company_name", "location_of_inc", "central_index_key"], keep=False
             ))][["company_name", "location_of_inc", "central_index_key", "parent_company_cik"]]

,company_name,location_of_inc,central_index_key,parent_company_cik
